# Analyse Comparative des Artistes Musicaux : Une Exploration en Profondeur

L'analyse présentée dans ce notebook s'appuie sur un jeu de données issues de la plateforme Hugging Face, contenant des informations détaillées sur les chansons de divers artistes, notamment leurs paroles, les artistes, les années de sortie, les styles musicaux, et d'autres métadonnées. Le fichier source, accessible à l'adresse https://huggingface.co/spaces/alihmaou/parolesparolesparoles/resolve/main/genius_top_50_fr_rap_pop_rock_analyzed.parquet, offre une mine de données pour explorer les tendances et caractéristiques des chansons dans le paysage musical contemporain. L'objectif de cette analyse est de préparer les données pour un diagramme radar permettant de comparer les artistes sur la base de plusieurs indicateurs.

Ces indicateurs incluent la vulgarité, l'engagement, la complexité lexicale et le nombre de mots dans les chansons. En calculant les moyennes de ces indicateurs par artiste et en les normalisant sur une échelle commune, nous visons à dresser un portrait comparatif des artistes. Cela facilitera la compréhension de leurs styles et approches dans la création musicale. L'analyse s'inscrit dans une démarche d'exploration de données musicales, cherchant à mettre en lumière les caractéristiques qui définissent les différents artistes et leurs œuvres.


## Méthodologie

La méthodologie adoptée dans cette analyse repose sur plusieurs étapes clés. Premièrement, nous avons utilisé DuckDB pour traiter les données stockées dans le fichier Parquet. Une requête SQL a été élaborée pour calculer les valeurs initiales pour chaque chanson en termes de vulgarité, engagement, complexité lexicale et nombre de mots. Ces valeurs ont ensuite été normalisées sur une échelle de 1 à 5 pour permettre une comparaison équitable entre les différents indicateurs. Par la suite, nous avons calculé les moyennes de ces indicateurs par artiste. Les données résultantes ont été utilisées pour créer un diagramme radar interactif à l'aide de Plotly, permettant de visualiser les caractéristiques de chaque artiste de manière comparative. Cette approche nous a permis de sélectionner les artistes avec le plus d'observations et de les représenter sur le diagramme radar.

## 🔧 Configuration

In [1]:
# Installation et imports
import duckdb as ddb
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

## 🦆 Chargement du dataset avec Duckdb

In [2]:
# Fonction de chargement complète (basée sur load_file_from_url_lite)
def load_file_from_url_lite(url_dataset="", loader="read_csv_auto", options="", nom_table="loaded_dataset", safe_mode=False):
    ddb.execute("install spatial")
    ddb.execute("load spatial")
    ddb.execute("INSTALL h3 FROM community")
    ddb.execute("LOAD h3")
    ddb.execute("install webbed from community;")
    ddb.execute("load webbed")
    ddb.execute("set force_download=True")
    ddb.execute(f"drop table if exists {nom_table}")   
    
    # Détection automatique du type de fichier
    if 'csv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'tsv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'txt' in url_dataset: 
        loader = "read_csv_auto"
    elif 'parquet' in url_dataset: 
        loader = "read_parquet"
    elif 'json' in url_dataset: 
        loader = "read_json_auto"
    elif 'xls' in url_dataset or 'xlsx' in url_dataset: 
        loader = "st_read"
    elif 'shp' in url_dataset: 
        loader = "st_read"
    elif 'geojson' in url_dataset: 
        loader = "st_read"
    elif 'xml' in url_dataset: 
        loader = "read_xml"
    elif 'html' in url_dataset: 
        loader = "read_html"
    else: 
        raise ValueError(f"Type de fichier non supporté pour {url_dataset}")
    
    if options=="": 
        options = "" 
    if 'csv' in url_dataset and safe_mode==True: 
        options = ", all_varchar=1" 
    if nom_table=="": 
        nom_table = "loaded_dataset"
    
    try:
        status = ddb.sql(f"""
            create or replace table {nom_table} as select *
            from
            {loader}("{url_dataset}" {options})
        """)
        return status
    except Exception as e:
        return f"Erreur au chargement du fichier : {str(e)}"

def run_query(sql):
    return ddb.sql(sql.replace("`"," ")).to_df()

# Chargement des données
load_file_from_url_lite("https://huggingface.co/spaces/alihmaou/parolesparolesparoles/resolve/main/genius_top_50_fr_rap_pop_rock_analyzed.parquet", safe_mode=True)
print("✅ Données chargées avec succès")

✅ Données chargées avec succès


## 🔍 Analyse SQL

Cette requête utilise des techniques SQL pour extraire et transformer les données de manière efficace.

In [3]:
# Exécution de la requête
df = run_query(""" WITH 
  -- Calculer les valeurs initiales pour chaque chanson
  chanson AS (
    SELECT 
      "artiste",
      CAST("vulgarite" AS DOUBLE) AS vulgarite,
      CASE 
        WHEN "message_politique" = 'élevé' THEN 5
        WHEN "message_politique" = 'fort' THEN 5
        WHEN "message_politique" = 'moyen' THEN 3
        WHEN "message_politique" = 'faible' THEN 1
        ELSE 1
      END AS engagement,
      CASE 
        WHEN "complexite_lexicale" = 'élevé' THEN 5
        WHEN "complexite_lexicale" = 'moyen' THEN 3
        WHEN "complexite_lexicale" = 'faible' THEN 1
        ELSE 1
      END AS complexite_lexicale,
      CAST("nombre_mots" AS DOUBLE) AS nombre_mots
    FROM loaded_dataset
  ),
  -- Normaliser les indicateurs sur une échelle de 1 à 5
  indicateurs AS (
    SELECT 
      "artiste",
      -- Vulgarité est déjà sur une échelle de 0 à 10, on ajuste à 1-5
      (vulgarite / 2) + 0.5 AS vulgarite,
      engagement,
      complexite_lexicale,
      -- Normaliser nombre_mots  supposons que min=0 et max=300 (selon aperçu)
      LEAST(5, GREATEST(1, (nombre_mots / 60) + 1)) AS nombre_mots
    FROM chanson
  ),
  -- Calculer les moyennes par artiste
  moyennes_artistes AS (
    SELECT 
      "artiste",
      AVG(vulgarite) AS vulgarite_moy,
      AVG(engagement) AS engagement_moy,
      AVG(complexite_lexicale) AS complexite_lexicale_moy,
      AVG(nombre_mots) AS nombre_mots_moy
    FROM indicateurs
    GROUP BY "artiste"
  )
SELECT 
  "artiste",
  vulgarite_moy,
  engagement_moy,
  complexite_lexicale_moy,
  nombre_mots_moy
FROM moyennes_artistes
ORDER BY "artiste" """)
print(f"Résultats : {len(df)} lignes")
df.head()

Résultats : 1281 lignes


,artiste,vulgarite_moy,engagement_moy,complexite_lexicale_moy,nombre_mots_moy
0,-M-,0.500000,1.000000,3.000000,3.05000
1,-M- (FRA),0.666667,1.222222,2.777778,3.37963
2,113,2.200000,2.800000,3.000000,5.00000
3,13 Organis,3.500000,3.000000,3.000000,5.00000
4,1PLIK140,4.000000,2.000000,3.000000,5.00000


## 📈 Visualisation

La bibliothèque principale utilisée est Plotly, qui est idéale pour créer des visualisations de données interactives et dynamiques. Le diagramme radar choisi permet de comparer les artistes sur plusieurs critères de style de manière claire et visuelle. Cela convient parfaitement pour une analyse comparative des caractéristiques des artistes musicaux.

In [4]:
import pandas as pd
import duckdb as ddb
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Préparation des données
df_artiste = df.groupby('artiste')[['vulgarite_moy', 'engagement_moy', 'complexite_lexicale_moy', 'nombre_mots_moy']].mean().reset_index()

# Normalisation des valeurs entre 1 et 5
for col in ['vulgarite_moy', 'engagement_moy', 'complexite_lexicale_moy', 'nombre_mots_moy']:
    df_artiste[col] = ((df_artiste[col] - df_artiste[col].min()) / (df_artiste[col].max() - df_artiste[col].min())) * 4 + 1

categories = ['Vulgarité', 'Engagement', 'Complexité lexicale', 'Nombre de mots']
max_artists = 10

# Sélection des artistes avec les plus d'observations
top_artists = df['artiste'].value_counts().nlargest(max_artists).index
df_filtered = df_artiste[df_artiste['artiste'].isin(top_artists)]

# Création du diagramme radar interactif
colors = px.colors.qualitative.Set3

dataviz = go.Figure()

for idx, artiste in enumerate(df_filtered['artiste']):
    values = df_filtered[df_filtered['artiste'] == artiste][['vulgarite_moy', 'engagement_moy', 'complexite_lexicale_moy', 'nombre_mots_moy']].values.flatten()
    
    dataviz.add_trace(go.Scatterpolar(
        r=values,
        theta=categories,
        fill='toself',
        name=artiste,
        line_color=colors[idx % len(colors)],
        fillcolor=colors[idx % len(colors)],
        opacity=0.6,
        hovertemplate='<b>%{fullData.name}</b><br>' +
                      '%{theta}: %{r:.2f}<br>' +
                      '<extra></extra>'
    ))

dataviz.update_layout(
    title=dict(
        text="Analyse comparative des artistes musicaux : Diagramme radar des indicateurs de style",
        x=0.5,
        font=dict(size=20, family="Arial Black")
    ),
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[1, 5],
            tickfont=dict(size=12)
        ),
        angularaxis=dict(
            tickfont=dict(size=14, family="Arial Black")
        )
    ),
    showlegend=True,
    legend=dict(
        x=1.1,
        y=0.5,
        font=dict(size=12)
    ),
    margin=dict(l=50, r=150, t=100, b=50),
    width=800,
    height=600
)

# Ajout d'annotation interactive
dataviz.update_annotations()
dataviz

---
*Made with ❤️ and with [duckit.fr](https://duckit.fr) - [Ali Hmaou](https://www.linkedin.com/in/ali-hmaou-6b7b73146/)*

In [5]:

# --- Variables injectées par le script ---
FINAL_OBJECT_VARIABLE_NAME = 'dataviz'
OUTPUT_IMAGE_NAME = 'published\\notebooks\\duckit_analysis_20250815_225320.png'
OUTPUT_HTML_NAME = 'published\\notebooks\\duckit_analysis_20250815_225320.html'

# ===================================================================
# CELLULE INJECTÉE AUTOMATIQUEMENT (VERSION ROBUSTE)
# ===================================================================
import sys
import os
# On importe les modules nécessaires pour l'export au cas où
try:
    from bokeh.io import save as bokeh_save
except ImportError:
    bokeh_save = None

try:
    # On s'assure que le dossier de sortie existe
    output_dir = os.path.dirname(OUTPUT_IMAGE_NAME)
    if output_dir:
        os.makedirs(output_dir, exist_ok=True)

    # On utilise globals().get() pour une récupération plus sûre
    final_object = globals().get(FINAL_OBJECT_VARIABLE_NAME)

    if final_object is None:
        # On lève une NameError pour être cohérent avec le code original
        raise NameError(f"name '{FINAL_OBJECT_VARIABLE_NAME}' is not defined")

    print(f"INFO: Variable '{FINAL_OBJECT_VARIABLE_NAME}' trouvée. Tentative d'exportation...")

    object_type = str(type(final_object))

    if 'plotly.graph_objs._figure.Figure' in object_type:
        print(f"--> Détecté : Plotly. Sauvegarde HTML et PNG.")
        # 1. Sauvegarde HTML pour l'interactivité
        print(f"--> Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        final_object.write_html(OUTPUT_HTML_NAME, include_plotlyjs='cdn')
        # 2. Sauvegarde PNG pour l'aperçu statique
        try:
            print(f"--> Tentative de sauvegarde PNG directe dans : {OUTPUT_IMAGE_NAME}")
            final_object.write_image(OUTPUT_IMAGE_NAME, scale=3, width=1200, height=800)
            print(f"--> Image Plotly sauvegardée avec succès.")
        except Exception as e:
            print(f"AVERTISSEMENT: La sauvegarde directe en PNG a échoué (kaleido est-il installé?).", file=sys.stderr)
            print(f"   Erreur: {e}", file=sys.stderr)
            print(f"--> PLAN B: On va utiliser la capture d'écran du HTML à la place.")
            # On crée un fichier marqueur pour que le script de post-traitement prenne le relais
            with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
                f.write("plotly")
    elif 'folium.folium.Map' in object_type:
        print(f"--> Détecté : Folium. Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        final_object.save(OUTPUT_HTML_NAME)
        # On crée un fichier marqueur générique pour la capture d'écran
        print(f"--> Création du marqueur de capture d'écran.")
        with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
            f.write("folium")
    elif 'altair.vegalite' in object_type and hasattr(final_object, 'save'):
        print(f"--> Détecté : Altair. Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        final_object.save(OUTPUT_HTML_NAME)
        # On crée un fichier marqueur générique pour la capture d'écran
        print(f"--> Création du marqueur de capture d'écran.")
        with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
            f.write("altair")
    elif 'bokeh.plotting' in object_type and bokeh_save is not None:
        print(f"--> Détecté : Bokeh. Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        bokeh_save(final_object, filename=OUTPUT_HTML_NAME, title="")
        # On crée un fichier marqueur générique pour la capture d'écran
        print(f"--> Création du marqueur de capture d'écran.")
        with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
            f.write("bokeh")
    elif 'matplotlib.figure.Figure' in object_type:
        print(f"--> Détecté : Matplotlib. Sauvegarde dans : {OUTPUT_IMAGE_NAME}")
        final_object.savefig(OUTPUT_IMAGE_NAME, dpi=300, bbox_inches='tight')
    else:
        print(f"AVERTISSEMENT: Type non supporté : {object_type}", file=sys.stderr)
except NameError:
    print(f"AVERTISSEMENT: Aucune variable '{FINAL_OBJECT_VARIABLE_NAME}' trouvée.", file=sys.stderr)
except Exception as e:
    print(f"ERREUR lors de l'exportation : {e}", file=sys.stderr)


INFO: Variable 'dataviz' trouvée. Tentative d'exportation...
--> Détecté : Plotly. Sauvegarde HTML et PNG.
--> Sauvegarde HTML dans : published\notebooks\duckit_analysis_20250815_225320.html
--> Tentative de sauvegarde PNG directe dans : published\notebooks\duckit_analysis_20250815_225320.png


--> Image Plotly sauvegardée avec succès.
